<a href="https://colab.research.google.com/github/SLy-Precision/Player-Detection-and-Team-Performance-Prediction/blob/main/00_Data_Analysis_Extraction_Evaluation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐾 🐆 00 Data Analysis/Extraction/Evaluation 🇲🇽 ⚽︎

### - Antonio Guerrero

## Extraction of all events in all seasons in liga MX 🔥

In [78]:
# Statsbomb api and more
!pip install --quiet statsbombpy
!pip install --quiet mplsoccer
!pip install --quiet highlight_text

# Libraries for the data manipulation and plot
from statsbombpy import sb
from tqdm import tqdm
import pandas as pd
from mplsoccer import VerticalPitch, Pitch, Sbapi
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns
import numpy as np

In [31]:
# OFFICIAL AND STATIC EVENT MAPPING DICTIONARY
# Based on the official StatsBomb API documentation

STATS_BOMB_EVENT_MAP = {
    '50/50': 33,
    'Ball Receipt*': 42,
    'Ball Recovery': 2,
    'Bad Behaviour': 24,
    'Block': 6,
    'Carry': 43,
    'Clearance': 9,
    'Dispossessed': 3,
    'Dribble': 14,
    'Dribbled Past': 39,
    'Duel': 4,
    'Error': 37,
    'Foul Committed': 22,
    'Foul Won': 21,
    'Goal Keeper': 23,
    'Half End': 34,
    'Half Start': 18,
    'Injury Stoppage': 40,
    'Interception': 10,
    'Miscontrol': 38,
    'Offside': 8,
    'Pass': 30,
    'Player Off': 27,
    'Player On': 26,
    'Pressure': 17,
    'Referee Ball-Drop': 41,
    'Shield': 28,
    'Shot': 16,
    'Starting XI': 35,
    'Substitution': 19,
    'Tactical Shift': 36,
    'Camera On*': 5,
    'Own Goal Against': 20,
    'Own Goal For': 25
}

# Extraction Function (Use None in his call if you want all matches, especific number for n matches)

def process_season_dynamically(competition_id, season_id, creds, output_filename, num_matches=None):
    """
    Extracts and processes events for a season using a static and official event map.
    """
    print(f"--- Starting extraction for season ID: {season_id} ---")
    try:
        matches = sb.matches(competition_id=competition_id, season_id=season_id, creds=creds)
        if num_matches is not None and num_matches > 0:
            matches_to_process = matches.head(num_matches)
        else:
            matches_to_process = matches

        all_events_raw = []
        for _, match_info in tqdm(matches_to_process.iterrows(), total=matches_to_process.shape[0], desc="Processing Matches"):
            match_id = match_info['match_id']
            try:
                df_events = sb.events(match_id=match_id, creds=creds)
                if df_events.empty: continue

                home_team_name = match_info['home_team']
                away_team_name = match_info['away_team']
                home_id = df_events[df_events['team'] == home_team_name]['team_id'].iloc[0]
                away_id = df_events[df_events['team'] == away_team_name]['team_id'].iloc[0]

                df_events = df_events.assign(
                    match_id=match_id, home_team_name=home_team_name, away_team_name=away_team_name,
                    home_team_id=home_id, away_team_id=away_id
                )
                all_events_raw.append(df_events)
            except Exception as e:
                print(f"\nWarning: Could not process events for match {match_id}. Error: {e}")

        if not all_events_raw: return None
        df_full_season = pd.concat(all_events_raw, ignore_index=True)
        df_full_season = df_full_season[df_full_season['type'] != 'Starting XI'].copy()

        # --- Use the static ---
        df_full_season['EventTypeId'] = df_full_season['type'].map(STATS_BOMB_EVENT_MAP)

        print("\nApplying transformations...")
        df = df_full_season.sort_values(by=['match_id', 'period', 'timestamp']).reset_index(drop=True)

        # Cleaning and creating columns
        df['timestamp_dt'] = pd.to_datetime(df['timestamp'], format='%H:%M:%S.%f', errors='coerce')
        df['eventSec'] = df['timestamp_dt'].dt.hour * 3600 + df['timestamp_dt'].dt.minute * 60 + df['timestamp_dt'].dt.second + df['timestamp_dt'].dt.microsecond / 1e6
        df['x_coord'] = df['location'].apply(lambda loc: loc[0] if isinstance(loc, list) and len(loc) > 0 else np.nan)
        df['y_coord'] = df['location'].apply(lambda loc: loc[1] if isinstance(loc, list) and len(loc) > 1 else np.nan)
        df['X'] = (df['x_coord'] / 120) * 100; df['Y'] = (df['y_coord'] / 80) * 100
        df['IsHomeTeam'] = (df['team'] == df['home_team_name']).astype(int)
        df['IsGoal'] = (df['shot_outcome'] == 'Goal').astype(int)

        def calculate_is_accurate(row):
            if row['type'] == 'Pass' and pd.isna(row['pass_outcome']): return 1
            if row['type'] == 'Shot' and row['shot_outcome'] in ['Goal', 'Saved', 'Post']: return 1
            if row['type'] == 'Dribble' and row['dribble_outcome'] == 'Complete': return 1
            return 0
        df['IsAccurate'] = df.apply(calculate_is_accurate, axis=1)

        df['HomeGoal'] = (df['IsGoal'] == 1) & (df['IsHomeTeam'] == 1); df['AwayGoal'] = (df['IsGoal'] == 1) & (df['IsHomeTeam'] == 0)
        df['HomeScore'] = df.groupby('match_id')['HomeGoal'].cumsum().shift(1).fillna(0)
        df['AwayScore'] = df.groupby('match_id')['AwayGoal'].cumsum().shift(1).fillna(0)
        df['TimeDelta'] = df.groupby(['match_id', 'period'])['eventSec'].diff().fillna(0)

        possession_won_conditions = [
            (df['type'] == 'Duel') & (df['duel_outcome'].isin(['Won', 'Success In Play'])),
            (df['type'] == 'Interception') & (df['interception_outcome'].isin(['Won', 'Success In Play'])),
            (df['type'] == 'Ball Recovery')
        ]
        df['IsPossessionWinningAction'] = np.select(possession_won_conditions, [1, 1, 1], default=0)
        df['IsDribbledPast'] = (df['type'] == 'Dribbled Past').astype(int)

        optional_metrics = ['shot_statsbomb_xg', 'obv_total_net', 'shot_gk_save_difficulty_xg', 'shot_gk_shot_stopping_xg_suppression']
        for metric in optional_metrics:
            if metric not in df.columns: df[metric] = np.nan
        for flag_col in ['under_pressure', 'counterpress']:
            if flag_col not in df.columns:
                df[flag_col] = False
            df[flag_col] = df[flag_col].fillna(False).astype(int)

        if 'shot_statsbomb_xg' in df.columns and 'pass_assisted_shot_id' in df.columns:
            shot_xg_map = df[df['type'] == 'Shot'].set_index('id')['shot_statsbomb_xg'].to_dict()
            df['xA'] = df['pass_assisted_shot_id'].map(shot_xg_map)

        final_columns = [
            'matchId', 'playerId', 'home_team_id', 'away_team_id', 'home_team_name', 'away_team_name',
            'eventSec', 'EventTypeId', 'period', 'minute', 'X', 'Y', 'IsHomeTeam',
            'IsAccurate', 'IsGoal', 'HomeScore', 'AwayScore', 'TimeDelta',
            'under_pressure', 'counterpress',
            'IsPossessionWinningAction', 'IsDribbledPast',
            'statsbomb_xg', 'xA', 'obv_total_net',
            'gk_save_difficulty_xg', 'gk_xg_prevented'
        ]

        df = df.rename(columns={
            'match_id': 'matchId', 'player_id': 'playerId',
            'shot_statsbomb_xg': 'statsbomb_xg',
            'shot_gk_save_difficulty_xg': 'gk_save_difficulty_xg',
            'shot_gk_shot_stopping_xg_suppression': 'gk_xg_prevented'
        })

        for col in final_columns:
            if col not in df.columns: df[col] = np.nan
        df_final = df[final_columns].copy()

        df_final.to_csv(output_filename, index=False)
        print(f"\n ✅ Process for season {season_id} completed. Data saved to '{output_filename} ❤️'")
        return df_final
    except Exception as e:
        print(f"\n ❌ Fatal error processing season {season_id}: {e} ❌")
        return None

In [32]:
# First season

# Credentials from ITAM (thanks)
user = "itam_hackathon@hudl.com"
password = "pGwIprel"
creds = {"user": user, "passwd": password}

# Process the whole season
df_2021_2022 = process_season_dynamically(
    competition_id=73,
    season_id=108,
    creds=creds,
    output_filename="Temporada_Completa_2021-2022.csv",
    num_matches=None
)

--- Starting extraction for season ID: 108 ---


Processing Matches:   0%|          | 0/342 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Processing Matches:   0%|          | 1/342 [00:03<18:54,  3.33s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Pr


Applying transformations...


/tmp/ipython-input-147298578.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_col] = df[flag_col].fillna(False).astype(int)
/tmp/ipython-input-147298578.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_col] = df[flag_col].fillna(False).astype(int)



 ✅ Process for season 108 completed. Data saved to 'Temporada_Completa_2021-2022.csv ❤️'


In [33]:
#Print the extraction
df_2021_2022.head()

,matchId,playerId,home_team_id,away_team_id,home_team_name,away_team_name,eventSec,EventTypeId,period,minute,X,Y,IsHomeTeam,IsAccurate,IsGoal,HomeScore,AwayScore,TimeDelta,under_pressure,counterpress,IsPossessionWinningAction,IsDribbledPast,statsbomb_xg,xA,obv_total_net,gk_save_difficulty_xg,gk_xg_prevented
0,3799351,NaN,1297,1287,Guadalajara,Atlético San Luis,0.000,18,1,0,NaN,NaN,1,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,3799351,NaN,1297,1287,Guadalajara,Atlético San Luis,0.000,18,1,0,NaN,NaN,0,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,3799351,26311.0,1297,1287,Guadalajara,Atlético San Luis,1.057,30,1,0,50.000000,50.000,1,1,0,0.0,0.0,1.057,0,0,0,0,NaN,NaN,0.005515,NaN,NaN
3,3799351,75306.0,1297,1287,Guadalajara,Atlético San Luis,2.349,42,1,0,37.166667,37.375,1,0,0,0.0,0.0,1.292,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,3799351,75306.0,1297,1287,Guadalajara,Atlético San Luis,2.349,43,1,0,37.166667,37.375,1,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,0.000413,NaN,NaN


In [35]:
# prnt sum of unique ids in the df, so we see how many games are
print(df_2021_2022['matchId'].nunique())


342


In [36]:
# Second Season
df_2022_2023 = process_season_dynamically(
    competition_id=73,
    season_id=235,
    creds=creds,
    output_filename="Temporada_Completa_2022-2023.csv",
    num_matches=None
)

--- Starting extraction for season ID: 235 ---


Processing Matches:   0%|          | 0/342 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Processing Matches:   0%|          | 1/342 [00:03<19:04,  3.36s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Pr


Applying transformations...


/tmp/ipython-input-147298578.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_col] = df[flag_col].fillna(False).astype(int)
/tmp/ipython-input-147298578.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_col] = df[flag_col].fillna(False).astype(int)



 ✅ Process for season 235 completed. Data saved to 'Temporada_Completa_2022-2023.csv ❤️'


In [37]:
#Print extraction
df_2022_2023.head()

,matchId,playerId,home_team_id,away_team_id,home_team_name,away_team_name,eventSec,EventTypeId,period,minute,X,Y,IsHomeTeam,IsAccurate,IsGoal,HomeScore,AwayScore,TimeDelta,under_pressure,counterpress,IsPossessionWinningAction,IsDribbledPast,statsbomb_xg,xA,obv_total_net,gk_save_difficulty_xg,gk_xg_prevented
0,3835101,NaN,1221,1227,Necaxa,Toluca,0.000,18,1,0,NaN,NaN,1,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,3835101,NaN,1221,1227,Necaxa,Toluca,0.000,18,1,0,NaN,NaN,0,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,3835101,28521.0,1221,1227,Necaxa,Toluca,0.205,30,1,0,50.000000,50.000,0,1,0,0.0,0.0,0.205,0,0,0,0,NaN,NaN,0.010836,NaN,NaN
3,3835101,27961.0,1221,1227,Necaxa,Toluca,0.405,42,1,0,54.083333,43.375,0,0,0,0.0,0.0,0.200,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,3835101,30701.0,1221,1227,Necaxa,Toluca,2.472,17,1,0,42.333333,60.500,1,0,0,0.0,0.0,2.067,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [38]:
# prnt sum of unique ids in the df, so we see how many games are
print(df_2022_2023['matchId'].nunique())

342


In [39]:
# Third season
df_2023_2024 = process_season_dynamically(
    competition_id=73,
    season_id=281,
    creds=creds,
    output_filename="Temporada_Completa_2023-2024.csv",
    num_matches=None
    )

--- Starting extraction for season ID: 281 ---


Processing Matches:   0%|          | 0/340 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Processing Matches:   0%|          | 1/340 [00:03<22:15,  3.94s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Pr


Applying transformations...


/tmp/ipython-input-147298578.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_col] = df[flag_col].fillna(False).astype(int)
/tmp/ipython-input-147298578.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_col] = df[flag_col].fillna(False).astype(int)



 ✅ Process for season 281 completed. Data saved to 'Temporada_Completa_2023-2024.csv ❤️'


In [40]:
#Print extraction
df_2023_2024.head()

,matchId,playerId,home_team_id,away_team_id,home_team_name,away_team_name,eventSec,EventTypeId,period,minute,X,Y,IsHomeTeam,IsAccurate,IsGoal,HomeScore,AwayScore,TimeDelta,under_pressure,counterpress,IsPossessionWinningAction,IsDribbledPast,statsbomb_xg,xA,obv_total_net,gk_save_difficulty_xg,gk_xg_prevented
0,3889637,NaN,1229,1291,América,Juárez,0.000,18,1,0,NaN,NaN,1,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,3889637,NaN,1229,1291,América,Juárez,0.000,18,1,0,NaN,NaN,0,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,3889637,35544.0,1229,1291,América,Juárez,0.044,30,1,0,50.833333,50.125,1,1,0,0.0,0.0,0.044,0,0,0,0,NaN,NaN,7.979107e-03,NaN,NaN
3,3889637,6357.0,1229,1291,América,Juárez,2.268,42,1,0,39.833333,46.000,1,0,0,0.0,0.0,2.224,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,3889637,6357.0,1229,1291,América,Juárez,2.598,30,1,0,39.666667,46.750,1,1,0,0.0,0.0,0.330,0,0,0,0,NaN,NaN,5.504000e-07,NaN,NaN


In [41]:
# prnt sum of unique ids in the df, so we see how many games are
print(df_2023_2024['matchId'].nunique())

340


In [42]:
# Fourth season
df_2024_2025 = process_season_dynamically(
    competition_id=73,
    season_id=317,
    creds=creds,
    output_filename="Temporada_Completa_2024-2025.csv",
    num_matches=None
)

--- Starting extraction for season ID: 317 ---


Processing Matches:   0%|          | 0/340 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Processing Matches:   0%|          | 1/340 [00:03<21:23,  3.79s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Pr


Applying transformations...


/tmp/ipython-input-147298578.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_col] = df[flag_col].fillna(False).astype(int)
/tmp/ipython-input-147298578.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_col] = df[flag_col].fillna(False).astype(int)



 ✅ Process for season 317 completed. Data saved to 'Temporada_Completa_2024-2025.csv ❤️'


In [43]:
#Print extraction
df_2024_2025.head()

,matchId,playerId,home_team_id,away_team_id,home_team_name,away_team_name,eventSec,EventTypeId,period,minute,X,Y,IsHomeTeam,IsAccurate,IsGoal,HomeScore,AwayScore,TimeDelta,under_pressure,counterpress,IsPossessionWinningAction,IsDribbledPast,statsbomb_xg,xA,obv_total_net,gk_save_difficulty_xg,gk_xg_prevented
0,3939816,NaN,1226,1299,Puebla,Santos Laguna,0.000,18,1,0,NaN,NaN,1,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,3939816,NaN,1226,1299,Puebla,Santos Laguna,0.000,18,1,0,NaN,NaN,0,0,0,0.0,0.0,0.000,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,3939816,37591.0,1226,1299,Puebla,Santos Laguna,0.477,30,1,0,50.000000,50.000,0,1,0,0.0,0.0,0.477,0,0,0,0,NaN,NaN,0.008289,NaN,NaN
3,3939816,37879.0,1226,1299,Puebla,Santos Laguna,1.769,42,1,0,44.250000,46.625,0,0,0,0.0,0.0,1.292,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,3939816,37879.0,1226,1299,Puebla,Santos Laguna,2.142,30,1,0,43.583333,48.250,0,1,0,0.0,0.0,0.373,0,0,0,0,NaN,NaN,-0.006251,NaN,NaN


In [44]:
# prnt sum of unique ids in the df, so we see how many games are
print(df_2024_2025['matchId'].nunique())

340


In [49]:
# Concatenate all the seasons in just one datset
df_liga_mx_completo = pd.concat([df_2021_2022, df_2022_2023, df_2023_2024, df_2024_2025], ignore_index=True)

df_liga_mx_completo.to_csv("Full_Dataset_2021-2025.csv", index=False)

In [50]:
# Printing total games
print(f"💪 We have a total games of: " + str(df_liga_mx_completo['matchId'].nunique()))

💪 We have a total games of: 1364


## Analysis and valdation that data is well structure and done 📊

In [79]:
# Lets call my unit in drive for the analysis of this dataset  🔵🔴🟡🟢
# (In case you're replying change this information for the full dataset)

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Proyect_Hackathon/'

df1 = pd.read_csv(path + 'Temporada_Completa_2021-2022.csv')
df2 = pd.read_csv(path + 'Temporada_Completa_2022-2023.csv')
df3 = pd.read_csv(path + 'Temporada_Completa_2023-2024.csv')
df4 = pd.read_csv(path + 'Temporada_Completa_2024-2025.csv')
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Lets get the array of the match ids

#Credentials for Statsbomb from ITAM (Again, thanks ITAM 🟢)
user = "itam_hackathon@hudl.com"
password = "pGwIprel"

# Save info
array_season_2021_2022 = sb.matches(competition_id=73, season_id=108, creds={"user": user, "passwd": password})['match_id'].to_list()
array_season_2022_2023 = sb.matches(competition_id=73, season_id=235, creds={"user": user, "passwd": password})['match_id'].to_list()
array_season_2023_2024 = sb.matches(competition_id=73, season_id=281, creds={"user": user, "passwd": password})['match_id'].to_list()
array_season_2024_2025 = sb.matches(competition_id=73, season_id=317, creds={"user": user, "passwd": password})['match_id'].to_list()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
# First thing first
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4442402 entries, 0 to 4442401
Data columns (total 27 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   matchId                    int64  
 1   playerId                   float64
 2   home_team_id               int64  
 3   away_team_id               int64  
 4   home_team_name             object 
 5   away_team_name             object 
 6   eventSec                   float64
 7   EventTypeId                int64  
 8   period                     int64  
 9   minute                     int64  
 10  X                          float64
 11  Y                          float64
 12  IsHomeTeam                 int64  
 13  IsAccurate                 int64  
 14  IsGoal                     int64  
 15  HomeScore                  float64
 16  AwayScore                  float64
 17  TimeDelta                  float64
 18  under_pressure             int64  
 19  counterpress               int64  
 20  Is

In [55]:
# 🧱 --- Basic structure validation --- 🧱
# Check if the DataFrame has missing or duplicated rows
print("Shape:", df.shape)
print("\nMissing values per column:")
print(df.isna().sum())

# Check for duplicated rows (should be 0 ideally)
duplicated_rows = df[df.duplicated()]
print(f"\nDuplicated rows found: {len(duplicated_rows)}")

# Conclusion: we see good results


Shape: (4442402, 27)

Missing values per column:
matchId                            0
playerId                       20278
home_team_id                       0
away_team_id                       0
home_team_name                     0
away_team_name                     0
eventSec                           0
EventTypeId                        0
period                             0
minute                             0
X                              42680
Y                              42680
IsHomeTeam                         0
IsAccurate                         0
IsGoal                             0
HomeScore                          0
AwayScore                          0
TimeDelta                          0
under_pressure                     0
counterpress                       0
IsPossessionWinningAction          0
IsDribbledPast                     0
statsbomb_xg                 4405881
xA                           4416051
obv_total_net                1946582
gk_save_difficulty_xg     

In [73]:
# ⏳ --- Temporal consistency checks --- ⏳
# 1. eventSec must be non-negative
neg_eventsec = df[df["eventSec"] < 0]
print(f"Negative eventSec values: {len(neg_eventsec)}")

# 2. minute must be between 0 and 120 (extra time upper bound)
invalid_minutes = df[(df["minute"] < 0) | (df["minute"] > 130)]
print(f"Invalid minute values: {len(invalid_minutes)}")

# 3. period must be within known match periods (1 to 4 (5 for penalties) for standard games)
invalid_periods = df[~df["period"].isin([1, 2, 3, 4,5])]
print(f"Invalid period values: {len(invalid_periods)}")

# 4. TimeDelta must not be negative
invalid_timedelta = df[df["TimeDelta"] < 0]
print(f"Negative TimeDelta values: {len(invalid_timedelta)}")

# 5. TimeDelta should not be extremely large (e.g., > 300 sec likely means missing events)
large_timedelta = df[df["TimeDelta"] > 300]
print(f"Suspiciously large TimeDelta values (>300s): {len(large_timedelta)}")

# Conclusion: we are seeing good results to think we have no trash data or noise


Negative eventSec values: 0
Invalid minute values: 0
Invalid period values: 0
Negative TimeDelta values: 0
Suspiciously large TimeDelta values (>300s): 77


In [57]:
# 🛰 --- Spatial coordinate validation --- 🛰
# Both X and Y should be between 0 and 100, representing a normalized field
invalid_xy = df[(df["X"] < 0) | (df["X"] > 100) | (df["Y"] < 0) | (df["Y"] > 100)]
print(f"Invalid X/Y coordinate values: {len(invalid_xy)}")

Invalid X/Y coordinate values: 6


In [58]:
# 🏷️ --- Team consistency checks --- 🏷️
# 1. Each matchId should have exactly 1 home_team_id and 1 away_team_id
team_inconsistency = (
    df.groupby("matchId")[["home_team_id", "away_team_id"]]
    .nunique()
    .reset_index()
)
invalid_matches = team_inconsistency[
    (team_inconsistency["home_team_id"] != 1)
    | (team_inconsistency["away_team_id"] != 1)
]
print(f"Matches with inconsistent team IDs: {len(invalid_matches)}")

# 2. Check that IsHomeTeam values are consistent with team IDs
# For each match, home_team_id events must have IsHomeTeam = 1
# and away_team_id events must have IsHomeTeam = 0
inconsistent_home_flags = []
for match_id, group in df.groupby("matchId"):
    home_id = group["home_team_id"].iloc[0]
    away_id = group["away_team_id"].iloc[0]
    invalid = group[
        ((group["IsHomeTeam"] == 1) & (group["playerId"].notna()) & (group["home_team_id"] != home_id))
        | ((group["IsHomeTeam"] == 0) & (group["away_team_id"] != away_id))
    ]
    if len(invalid) > 0:
        inconsistent_home_flags.append(match_id)
print(f"Matches with inconsistent IsHomeTeam flags: {len(inconsistent_home_flags)}")


Matches with inconsistent team IDs: 0
Matches with inconsistent IsHomeTeam flags: 0


In [74]:
# 📋 --- EventTypeId validation --- 📋
valid_event_ids = {2,3,4,5,6,8,9,10,14,16,17,18,19,20,21,22,23,24,25,26,27,28,30,33,34,35,36,37,38,39,40,41,42,43}
invalid_event_ids = df[~df["EventTypeId"].isin(valid_event_ids)]
print(f"Rows with invalid EventTypeId: {len(invalid_event_ids)}")

# Conclusion: Good ✅


Rows with invalid EventTypeId: 0


In [60]:
# 🧤 --- Logical consistency between performance metrics --- 🧤
# 1. xG values must be between 0 and 1
invalid_xg = df[(df["statsbomb_xg"] < 0) | (df["statsbomb_xg"] > 1)]
print(f"Invalid xG values (outside [0,1]): {len(invalid_xg)}")

# 2. xA should also be between 0 and 1 or NaN
invalid_xa = df[(df["xA"].notna()) & ((df["xA"] < 0) | (df["xA"] > 1))]
print(f"Invalid xA values (outside [0,1]): {len(invalid_xa)}")

# 3. gk_xg_prevented should usually be within [-5, +5] range
extreme_gk_prevented = df[(df["gk_xg_prevented"] < -5) | (df["gk_xg_prevented"] > 5)]
print(f"Unrealistic gk_xg_prevented values: {len(extreme_gk_prevented)}")

# 4. IsGoal = 1 should only appear for EventTypeId = 16 (Shot)
invalid_goals = df[(df["IsGoal"] == 1) & (df["EventTypeId"] != 16)]
print(f"Inconsistent IsGoal values: {len(invalid_goals)}")


Invalid xG values (outside [0,1]): 0
Invalid xA values (outside [0,1]): 0
Unrealistic gk_xg_prevented values: 0
Inconsistent IsGoal values: 0


In [61]:
# 🧠 --- Binary flag validation --- 🧠
binary_columns = [
    "IsHomeTeam", "IsAccurate", "IsGoal", "under_pressure",
    "counterpress", "IsPossessionWinningAction", "IsDribbledPast"
]
for col in binary_columns:
    invalid_vals = df[~df[col].isin([0, 1])]
    print(f"{col}: {len(invalid_vals)} invalid (non-binary) values")


IsHomeTeam: 0 invalid (non-binary) values
IsAccurate: 0 invalid (non-binary) values
IsGoal: 0 invalid (non-binary) values
under_pressure: 0 invalid (non-binary) values
counterpress: 0 invalid (non-binary) values
IsPossessionWinningAction: 0 invalid (non-binary) values
IsDribbledPast: 0 invalid (non-binary) values


In [62]:
# 📈 --- Score progression validation --- 📈
# Scores should be non-decreasing within each match for each team
invalid_scores = []
for match_id, group in df.groupby("matchId"):
    if not group["HomeScore"].is_monotonic_increasing or not group["AwayScore"].is_monotonic_increasing:
        invalid_scores.append(match_id)
print(f"Matches with inconsistent score progression: {len(invalid_scores)}")


Matches with inconsistent score progression: 1248


In [64]:
# ⚽️🥅 --- Score update logic validation (with own goals) --- ⚽️🥅
invalid_score_changes = []

for match_id, group in df.groupby("matchId"):
    group = group.sort_values(["period", "eventSec"]).reset_index(drop=True)

    prev_home = group["HomeScore"].iloc[0]
    prev_away = group["AwayScore"].iloc[0]

    for idx, row in group.iterrows():
        home, away = row["HomeScore"], row["AwayScore"]
        home_changed = home != prev_home
        away_changed = away != prev_away

        if home_changed or away_changed:
            # Valid normal goal condition
            valid_goal = (row["EventTypeId"] == 16 and row["IsGoal"] == 1)

            # Detect potential own goal
            is_own_goal = False
            if valid_goal:
                # Home team action but away score increases
                if row["IsHomeTeam"] == 1 and away > prev_away:
                    is_own_goal = True
                # Away team action but home score increases
                elif row["IsHomeTeam"] == 0 and home > prev_home:
                    is_own_goal = True

            # If score changes but not a valid goal or own goal, flag as invalid
            if not (valid_goal or is_own_goal):
                invalid_score_changes.append({
                    "matchId": match_id,
                    "index": idx,
                    "period": row["period"],
                    "minute": row["minute"],
                    "HomeScore_before": prev_home,
                    "AwayScore_before": prev_away,
                    "HomeScore_after": home,
                    "AwayScore_after": away,
                    "EventTypeId": row["EventTypeId"],
                    "IsGoal": row["IsGoal"],
                    "IsHomeTeam": row["IsHomeTeam"]
                })

        prev_home, prev_away = home, away

invalid_score_changes_df = pd.DataFrame(invalid_score_changes)
print(f"Invalid score changes (excluding valid goals and own goals): {len(invalid_score_changes_df)}")
print(invalid_score_changes_df.head(10))


Invalid score changes (excluding valid goals and own goals): 4889
   matchId  index  period  minute  HomeScore_before  AwayScore_before  HomeScore_after  AwayScore_after  EventTypeId  IsGoal  IsHomeTeam
0  3799351    427       1      11               0.0               0.0              0.0              1.0           23       0           1
1  3799351   2276       2      64               0.0               1.0              0.0              2.0           23       0           1
2  3799351   2536       2      78               0.0               2.0              1.0              2.0           23       0           0
3  3799352      1       1       0               1.0               2.0              0.0              0.0           18       0           0
4  3799352    734       1      23               0.0               0.0              0.0              1.0           23       0           1
5  3799352   1777       2      53               0.0               1.0              0.0              2.0         

In [63]:
# 🩺 --- Dataset health summary --- 🩺
summary = {
    "Missing values": df.isna().sum().sum(),
    "Negative TimeDelta": len(invalid_timedelta),
    "Invalid EventTypeId": len(invalid_event_ids),
    "Invalid X/Y": len(invalid_xy),
    "Inconsistent team IDs": len(invalid_matches),
    "Invalid xG": len(invalid_xg),
    "Invalid IsGoal flag": len(invalid_goals)
}
print("\n--- DATASET QUALITY SUMMARY ---")
for k, v in summary.items():
    print(f"{k}: {v}")



--- DATASET QUALITY SUMMARY ---
Missing values: 19735152
Negative TimeDelta: 0
Invalid EventTypeId: 210
Invalid X/Y: 6
Inconsistent team IDs: 0
Invalid xG: 0
Invalid IsGoal flag: 0


In [66]:
# --- Step 1: Sort events to preserve temporal order within each match ---
df_sorted = df.sort_values(["matchId", "period", "eventSec"]).reset_index(drop=True)

# --- Step 2: Identify where the score changes ---
df_sorted["score_change"] = (
    (df_sorted["HomeScore"].diff() != 0) | (df_sorted["AwayScore"].diff() != 0)
)

# We need to reset the comparison when match changes
df_sorted["match_change"] = df_sorted["matchId"].diff().fillna(0) != 0
df_sorted.loc[df_sorted["match_change"], "score_change"] = False

# --- Step 3: Collect context (2 events before each score change) ---
records = []

for match_id, group in df_sorted.groupby("matchId"):
    group = group.reset_index()
    for idx, row in group[group["score_change"]].iterrows():
        # Get two previous events (if available)
        prev_events = group.loc[max(0, idx-2):idx-1, ["EventTypeId", "IsGoal", "IsAccurate"]]
        context = {
            "matchId": match_id,
            "index": row["index"],
            "period": row["period"],
            "minute": row["minute"],
            "event_type_causing_change": row["EventTypeId"],
            "is_goal_flag": row["IsGoal"],
            "home_score_before": group.loc[idx-1, "HomeScore"] if idx > 0 else None,
            "away_score_before": group.loc[idx-1, "AwayScore"] if idx > 0 else None,
            "home_score_after": row["HomeScore"],
            "away_score_after": row["AwayScore"],
            "prev1_EventTypeId": prev_events["EventTypeId"].iloc[-1] if len(prev_events) > 0 else None,
            "prev1_IsGoal": prev_events["IsGoal"].iloc[-1] if len(prev_events) > 0 else None,
            "prev1_IsAccurate": prev_events["IsAccurate"].iloc[-1] if len(prev_events) > 0 else None,
            "prev2_EventTypeId": prev_events["EventTypeId"].iloc[0] if len(prev_events) == 2 else None,
            "prev2_IsGoal": prev_events["IsGoal"].iloc[0] if len(prev_events) == 2 else None,
            "prev2_IsAccurate": prev_events["IsAccurate"].iloc[0] if len(prev_events) == 2 else None
        }
        records.append(context)

score_context_df = pd.DataFrame(records)

print(f"Total score changes detected: {len(score_context_df)}")

# --- Step 4: Analyze patterns ---
# Count how many score changes were preceded by a Shot (16) or Goalkeeper Action (23)
preceded_by_shot = score_context_df[
    (score_context_df["prev1_EventTypeId"] == 16)
    | (score_context_df["prev2_EventTypeId"] == 16)
]

preceded_by_gk_action = score_context_df[
    (score_context_df["prev1_EventTypeId"] == 23)
    | (score_context_df["prev2_EventTypeId"] == 23)
]

print(f"\nScore changes preceded by a Shot: {len(preceded_by_shot)}")
print(f"Score changes preceded by a GK Action (23): {len(preceded_by_gk_action)}")

# --- Step 5: Inspect a few interesting samples ---
print("\nSample of score changes with goalkeepers just before:")
print(score_context_df[
    (score_context_df["prev1_EventTypeId"] == 23) |
    (score_context_df["prev2_EventTypeId"] == 23)
].head(10))


Total score changes detected: 4890

Score changes preceded by a Shot: 3641
Score changes preceded by a GK Action (23): 79

Sample of score changes with goalkeepers just before:
     matchId   index  period  minute  event_type_causing_change  is_goal_flag  home_score_before  away_score_before  home_score_after  away_score_after  prev1_EventTypeId  prev1_IsGoal  prev1_IsAccurate  prev2_EventTypeId  prev2_IsGoal  prev2_IsAccurate
56   3799369   54708       1      21                         23             0                0.0                0.0               1.0               0.0               16.0           1.0               1.0               23.0           0.0               0.0
166  3799406  166789       1      44                         23             0                0.0                3.0               0.0               4.0               16.0           1.0               1.0               23.0           0.0               0.0
279  3799443  278499       1      12                         

In [68]:
# --- Step 1: Sort events and detect score changes ---
df_sorted = df.sort_values(["matchId", "period", "eventSec"]).reset_index(drop=True)

df_sorted["score_change"] = (
    (df_sorted["HomeScore"].diff() != 0) | (df_sorted["AwayScore"].diff() != 0)
)
df_sorted["match_change"] = df_sorted["matchId"].diff().fillna(0) != 0
df_sorted.loc[df_sorted["match_change"], "score_change"] = False

# --- Step 2: Identify all score change events ---
score_changes = df_sorted[df_sorted["score_change"]].copy()

# --- Step 3: Mark those explained by "Shot" or "GK Action" ---
score_changes["explained_by_shot"] = False
score_changes["explained_by_gk"] = False

# Look two events back to capture sequences like "Shot -> GK -> Score change"
for match_id, group in df_sorted.groupby("matchId"):
    idxs = group.index.tolist()
    for i in range(2, len(group)):
        row = group.iloc[i]
        if not row["score_change"]:
            continue
        prev1, prev2 = group.iloc[i - 1], group.iloc[i - 2]

        # If one of the previous two events is a Shot (IsGoal=1)
        if (prev1["EventTypeId"] == 16 and prev1["IsGoal"] == 1) or \
           (prev2["EventTypeId"] == 16 and prev2["IsGoal"] == 1):
            score_changes.loc[row.name, "explained_by_shot"] = True

        # If one of the previous two events is a GK Action
        if (prev1["EventTypeId"] == 23) or (prev2["EventTypeId"] == 23):
            score_changes.loc[row.name, "explained_by_gk"] = True

# --- Step 4: Filter unexplained score changes ---
unexplained = score_changes[
    (~score_changes["explained_by_shot"]) &
    (~score_changes["explained_by_gk"])
]

print(f"Unexplained score changes: {len(unexplained)}")

# --- Step 5: See which event types cause these unexplained changes ---
summary = unexplained["EventTypeId"].value_counts().reset_index()
summary.columns = ["EventTypeId", "Count"]
summary["Percentage"] = (summary["Count"] / len(unexplained) * 100).round(2)

print("\n--- Breakdown of unexplained score change events ---")
print(summary)

# --- Step 6: Show manual inspection ---
print("\nSample unexplained events:")
cols = ["matchId", "period", "minute", "EventTypeId", "IsGoal", "IsAccurate",
        "HomeScore", "AwayScore", "X", "Y"]
print(unexplained[cols].head(15))


Unexplained score changes: 1249

--- Breakdown of unexplained score change events ---
   EventTypeId  Count  Percentage
0           18   1248       99.92
1            2      1        0.08

Sample unexplained events:
       matchId  period  minute  EventTypeId  IsGoal  IsAccurate  HomeScore  AwayScore   X   Y
0      3799351       1       0           18       0           0        0.0        0.0 NaN NaN
3067   3799352       1       0           18       0           0        0.0        0.0 NaN NaN
6188   3799353       1       0           18       0           0        0.0        0.0 NaN NaN
9031   3799354       1       0           18       0           0        0.0        0.0 NaN NaN
12361  3799355       1       0           18       0           0        0.0        0.0 NaN NaN
15284  3799356       1       0           18       0           0        0.0        0.0 NaN NaN
21886  3799358       1       0           18       0           0        0.0        0.0 NaN NaN
24613  3799359       1       0  

In [69]:
df[["statsbomb_xg", "xA", "obv_total_net"]].describe()

,statsbomb_xg,xA,obv_total_net
count,36521.000000,26351.000000,2.495820e+06
mean,0.097693,0.085393,1.560013e-03
std,0.145614,0.107158,3.722873e-02
min,0.000180,0.002404,-1.094597e+00
25%,0.025295,0.027333,-1.745542e-03
50%,0.048520,0.050646,3.329026e-04
75%,0.096621,0.095679,3.809206e-03
max,0.977087,0.977087,1.608129e+00


In [70]:
# Is accurate inly is 1 in 14,16,30 events

df.groupby("EventTypeId")["IsAccurate"].mean()

# Nice ✅


,IsAccurate
EventTypeId,
2,0.000000
3,0.000000
4,0.000000
6,0.000000
8,0.000000
9,0.000000
10,0.000000
14,0.521300
16,0.347280


In [71]:
# Anomaly in matches?

event_count = df["matchId"].value_counts()
event_count.describe()


,count
count,1364.000000
mean,3256.892962
std,333.264874
min,1962.000000
25%,3032.750000
50%,3253.500000
75%,3494.250000
max,4248.000000


In [72]:
# Distribution in events?

df["EventTypeId"].value_counts(normalize=True) * 100

# Not balanced (waited) ✅


,proportion
EventTypeId,
30,27.574204
42,25.413571
43,21.824635
17,9.372947
2,2.894583
4,2.132945
9,1.285476
6,1.112911
23,0.987371


In [76]:
# --- Clip coordinates to the 0-100 range to handle edge cases ---
df['X'] = np.clip(df['X'], 0, 100)
df['Y'] = np.clip(df['Y'], 0, 100)

In [77]:
# Again:

# 🛰 --- Spatial coordinate validation --- 🛰
# Both X and Y should be between 0 and 100, representing a normalized field
invalid_xy = df[(df["X"] < 0) | (df["X"] > 100) | (df["Y"] < 0) | (df["Y"] > 100)]
print(f"Invalid X/Y coordinate values: {len(invalid_xy)}")

# Now we have good results ✅

Invalid X/Y coordinate values: 0


## 👥 Extraction and verification of players ids with his name (or information) 👥

In [81]:
# 🏃🏻‍♂️ ⚽ --- Just to finish lets make the df with the player id and his name --- 🏃🏻‍♂️ ⚽

import pandas as pd
from statsbombpy import sb
from tqdm import tqdm

def create_player_map(competition_id, season_id, creds):
    """
    Creates a DataFrame with a unique list of players (ID and name) for a given season.
    This version correctly handles the dictionary returned by sb.lineups().
    """
    print(f"--- Creating player map for season ID: {season_id} ---")
    try:
        # 1. Get all matches for the season
        matches = sb.matches(competition_id=competition_id, season_id=season_id, creds=creds)
        match_ids = matches['match_id'].tolist()

        all_lineups = []
        # 2. Loop through each match to get the lineups
        for match_id in tqdm(match_ids, desc="Fetching Lineups"):
            # sb.lineups() returns a dictionary like {team_name: lineup_df}
            lineups_dict = sb.lineups(match_id=match_id, creds=creds)

            # --- CORRECTION: Add each DataFrame from the dictionary to our list ---
            for team_lineup_df in lineups_dict.values():
                all_lineups.append(team_lineup_df)

        # 3. Combine all lineup DataFrames into a single one
        df_all_players = pd.concat(all_lineups, ignore_index=True)

        # 4. Select relevant columns and remove duplicates
        df_players = df_all_players[['player_id', 'player_name']].copy()
        df_players.drop_duplicates(subset=['player_id'], inplace=True)

        # 5. Rename columns for consistency
        df_players.rename(columns={'player_id': 'playerId', 'player_name': 'playerName'}, inplace=True)

        print(f"\n✅ Player map created successfully with {len(df_players)} unique players.")
        return df_players

    except Exception as e:
        print(f"\n❌ Error creating player map: {e}")
        return None

# Notice we will have only player who played at least one time 👁️ 👄 👁️

df_jugadores1 = create_player_map(competition_id=73, season_id=108, creds=creds)


--- Creating player map for season ID: 108 ---


Fetching Lineups: 100%|██████████| 342/342 [07:24<00:00,  1.30s/it]


✅ Player map created successfully with 665 unique players.


In [82]:
df_jugadores1.head()

,playerId,playerName
0,26289,Javier Güemez López
1,26306,Adam Fernando Bareiro Gamarra
2,26331,Marcelo Barovero
3,27083,Jhon Fredy Duque Arias
4,27967,Unai Bilbao Arteta


In [83]:
df_jugadores2 = create_player_map(competition_id=73, season_id=235, creds=creds)

--- Creating player map for season ID: 235 ---


Fetching Lineups: 100%|██████████| 342/342 [07:22<00:00,  1.29s/it]



✅ Player map created successfully with 653 unique players.


In [89]:
df_jugadores2.head()

,playerId,playerName
0,6666,Sabin Merino Zuloaga
1,10846,Jhon Eduard Murillo Romaña
2,22208,Abel Mathías Hernández Platero
3,26289,Javier Güemez López
4,26328,Rubens Omar Óscar Sambueza


In [84]:
df_jugadores3 = create_player_map(competition_id=73, season_id=281, creds=creds)

--- Creating player map for season ID: 281 ---


Fetching Lineups: 100%|██████████| 340/340 [07:33<00:00,  1.33s/it]



✅ Player map created successfully with 661 unique players.


In [90]:
df_jugadores3.head()

,playerId,playerName
0,9926,Leonardo Bonatini Lohner Maia
1,10846,Jhon Eduard Murillo Romaña
2,26289,Javier Güemez López
3,26311,Ángel Zaldívar Caviedes
4,26327,Julio César Domínguez Juárez


In [85]:
df_jugadores4 = create_player_map(competition_id=73, season_id=317, creds=creds)

--- Creating player map for season ID: 317 ---


Fetching Lineups: 100%|██████████| 340/340 [07:34<00:00,  1.34s/it]



✅ Player map created successfully with 673 unique players.


In [91]:
df_jugadores4.head()

,playerId,playerName
0,10846,Jhon Eduard Murillo Romaña
1,26317,Adrián Mora Barraza
2,26324,Aldo Paul Rocha González
3,27567,Mateo Ezequiel García
4,28424,Edgar Zaldivar Valverde


In [92]:
df_jugadores_full = pd.concat([df_jugadores1, df_jugadores2, df_jugadores3, df_jugadores4], ignore_index=True)

### --- 👁️ Lastly, lets check coherence in this last df 👁️ ---

In [93]:
df_jugadores_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2652 entries, 0 to 2651
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   playerId    2652 non-null   int64 
 1   playerName  2652 non-null   object
dtypes: int64(1), object(1)
memory usage: 41.6+ KB


In [96]:
def validate_player_map(df_players):
    """
    Performs a series of data quality checks on the concatenated player map DataFrame.
    """
    print("--- 🕵️‍♂️ Validating Player Map DataFrame ---")

    # Check 1: Missing Values
    print("\n[Check 1] Looking for missing values...")
    missing_values = df_players.isna().sum()
    if missing_values.sum() == 0:
        print("✅ OK: No missing values found in playerId or playerName.")
    else:
        print("❌ WARNING: Missing values found!")
        print(missing_values)

    # Check 2: Duplicate Player IDs
    print("\n[Check 2] Looking for duplicate player IDs...")
    duplicate_ids = df_players['playerId'].duplicated().sum()
    if duplicate_ids == 0:
        print("✅ OK: All player IDs are unique.")
    else:
        print(f"❌ CRITICAL: Found {duplicate_ids} duplicate player IDs. This must be fixed.")

    # Check 3: Conflicting Player Names (same name, different IDs)
    print("\n[Check 3] Looking for conflicting player names...")
    # Group by name and count how many unique IDs are associated with it
    name_to_id_counts = df_players.groupby('playerName')['playerId'].nunique()
    conflicting_names = name_to_id_counts[name_to_id_counts > 1]

    if conflicting_names.empty:
        print("✅ OK: No player names are associated with multiple IDs.")
    else:
        print(f"🟡 INFO: Found {len(conflicting_names)} names associated with multiple player IDs.")
        print("This is often normal (e.g., two players named 'Adama Traore'), but requires review.")

    print("\n--- Validation Complete ---")

In [97]:
validate_player_map(df_jugadores_full)

--- 🕵️‍♂️ Validating Player Map DataFrame ---

[Check 1] Looking for missing values...
✅ OK: No missing values found in playerId or playerName.

[Check 2] Looking for duplicate player IDs...
❌ CRITICAL: Found 1418 duplicate player IDs. This must be fixed.

[Check 3] Looking for conflicting player names...
✅ OK: No player names are associated with multiple IDs.

--- Validation Complete ---


In [98]:
# Cleaning
df_jugadores_full = df_jugadores_full.drop_duplicates(subset=['playerId'], keep='first')

In [99]:
validate_player_map(df_jugadores_full)

--- 🕵️‍♂️ Validating Player Map DataFrame ---

[Check 1] Looking for missing values...
✅ OK: No missing values found in playerId or playerName.

[Check 2] Looking for duplicate player IDs...
✅ OK: All player IDs are unique.

[Check 3] Looking for conflicting player names...
✅ OK: No player names are associated with multiple IDs.

--- Validation Complete ---


In [101]:
df_jugadores_full.to_csv('df_players_info_full.csv', index=False)

In [109]:
def validate_goalkeepers(df_events):
    """
    Checks each match to validate the number of unique players performing 'Goal Keeper' events.
    """
    print("--- 🧤 Validating Goalkeeper Logic ---")

    # Filter for 'Goal Keeper' events only
    gk_events = df_events[df_events['EventTypeId'] == 23 ].copy()

    if gk_events.empty:
        print("🟡 INFO: No 'Goal Keeper' events found to validate.")
        return

    # Group by match and count unique players performing these actions
    gk_counts_per_match = gk_events.groupby('matchId')['playerId'].nunique()

    # Find matches that have an unusual number of goalkeepers (less than 2 or more than 4)
    unusual_matches = gk_counts_per_match[(gk_counts_per_match < 2) | (gk_counts_per_match > 4)]

    if unusual_matches.empty:
        print("✅ OK: All matches have a reasonable number of goalkeepers (2-4).")
        print("This suggests that 'Goal Keeper' events are correctly associated with specific players.")
    else:
        print("❌ WARNING: Found matches with an unusual number of goalkeepers.")
        print("This could indicate data errors or very rare situations (e.g., multiple GK substitutions).")
        print("Matches to review:")
        print(unusual_matches)

    print("\n--- Goalkeeper Validation Complete ---")

In [110]:
def validate_player_id_coverage(df_events, df_players):
    """
    Checks if all player IDs in the event data are present in the player map.
    """
    print("\n--- 🔗 Validating Player ID Coverage ---")

    # Get unique player IDs from both DataFrames
    event_player_ids = set(df_events['playerId'].dropna().unique())
    map_player_ids = set(df_players['playerId'].dropna().unique())

    # Find IDs that are in the events but NOT in the player map
    missing_ids_in_map = event_player_ids - map_player_ids

    if not missing_ids_in_map:
        print("✅ OK: All player IDs from the event data are present in the player map.")
    else:
        print(f"❌ CRITICAL: Found {len(missing_ids_in_map)} player IDs in the event data that are NOT in the player map.")
        print("This means the player map is incomplete.")
        print("Missing IDs:", missing_ids_in_map)

    print("\n--- Coverage Validation Complete ---")

In [111]:
# Lets do this last verification of integrity of the DATA checking the relation between this 2 dfs

validate_goalkeepers(df)
validate_player_id_coverage(df, df_jugadores_full)

--- 🧤 Validating Goalkeeper Logic ---
✅ OK: All matches have a reasonable number of goalkeepers (2-4).
This suggests that 'Goal Keeper' events are correctly associated with specific players.

--- Goalkeeper Validation Complete ---

--- 🔗 Validating Player ID Coverage ---
✅ OK: All player IDs from the event data are present in the player map.

--- Coverage Validation Complete ---
